In [ ]:
from setup_module import init_setup
import numpy as np
from classification_module import init_classification, combine_accuracy_graphs,prep_data,remove_irelevant



In [ ]:
dataset = init_setup()
dataset_prep = prep_data(dataset)

In [28]:

easy_mode = 1
if easy_mode == 1:
    easy_mods = [
        "OOK",
        "4ASK",
        "BPSK",
        "QPSK",
        "8PSK",
        "16QAM",
        "AM-SSB-SC",
        "AM-DSB-SC",
        "FM",
        "GMSK",
        "OQPSK",
    ]
    easy_mask = np.isin(dataset_prep['label'], easy_mods)
    labels = dataset_prep['label'][easy_mask]
    snr = dataset_prep['snr'][easy_mask]
    samples = dataset_prep['samples'][easy_mask]
    samples_dx = dataset_prep['dx'][easy_mask]
    sample_real = dataset_prep['real'][easy_mask]
    sample_imag = dataset_prep['imag'][easy_mask]
    # sample_amp = dataset_prep['amplitude'][easy_mask]
    # sample_phs = dataset_prep['phase'][easy_mask]
    
    group1 = ['AM-DSB-SC','AM-SSB-SC','FM','GMSK','OQPSK']
    group2 = ['OOK','4ASK','BPSK','QPSK','8PSK','16QAM']
    groups = [group1, group2]
else:
    group1 = [
        "OOK",
        "AM-SSB-WC",
        "AM-SSB-SC",
        "AM-DSB-WC",
        "AM-DSB-SC",
        "FM",
        "GMSK",
        "OQPSK",
    ]  # Low Order and Analog
    group2 = [
            "4ASK",
            "8ASK",
            "16QAM",
            "32QAM",
            "64QAM",
            "128QAM",
            "256QAM",
        ]  # ASK and QAM
    group3 = [
            "BPSK",
            "QPSK",
            "8PSK",
            "16PSK",
            "32PSK",
            "16APSK",
            "32APSK",
            "64APSK",
            "128APSK",
        ]  # PSK and APSK
    groups = [group1, group2, group3]
    labels = dataset_prep['label']
    snr = dataset_prep['snr']
    samples = dataset_prep['samples']
    samples_dx = dataset_prep['dx']
    sample_real = dataset_prep['real']
    sample_imag = dataset_prep['imag']
    # sample_amp = dataset_prep['ampllitude']
    # sample_phs = dataset_prep['phase']

samples_dx = remove_irelevant(samples_dx)
samples_with_dx = np.concatenate((samples,samples_dx),1)
samples_cum_dx_IQ = np.concatenate((samples,samples_dx,sample_real,sample_imag),1)


In [25]:

print("we are here")
train_tresh = 1
print("classification for cumulants...")
accuracy_data0 = init_classification(samples, labels, snr, "cumulants", groups, easy_mode, train_tresh)
accuracy_data1 = init_classification(samples_with_dx, labels, snr, "cumulants_and_dx", groups, easy_mode, train_tresh)
accuracy_data2 = init_classification(samples_cum_dx_IQ, labels, snr, "cumulants_with_dx_IQ", groups, easy_mode, train_tresh)

we are here
classification for cumulants...
change
Depth of the decision tree: 67
Classify per SNR
Classify per Label
saved plots
change
Depth of the decision tree: 43
Classify per SNR
Classify per Label
saved plots
change
Depth of the decision tree: 41
Classify per SNR
Classify per Label
saved plots


In [26]:
combine_accuracy_graphs(accuracy_data0,accuracy_data1,accuracy_data2)